<a href="https://colab.research.google.com/github/ErickMMuniz/NetworkAnalysis_TwitterTrends/blob/main/colab_burstkit_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download all datasets and codes

In [23]:
!sh -f bin_colab.sh

mkdir: cannot create directory ‘fooo/bar’: No such file or directory


In [1]:
#@title Dataset

!wget https://zenodo.org/record/4686013/files/virality2013.tar.gz
!mkdir data
!mkdir data/virality2013.tar/
!tar -xf virality2013.tar.gz -C data/virality2013.tar/

--2022-06-08 00:24:29--  https://zenodo.org/record/4686013/files/virality2013.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 526346240 (502M) [application/octet-stream]
Saving to: ‘virality2013.tar.gz’

virality2013.tar.gz 100%[===================>] 501.96M  14.0MB/s    in 46s     

2022-06-08 00:25:16 (10.9 MB/s) - ‘virality2013.tar.gz’ saved [526346240/526346240]



In [2]:
#@title clone repository
!git clone https://github.com/ErickMM98/NetworkAnalysis_TwitterTrends.git
!mv /content/NetworkAnalysis_TwitterTrends/burstkit /content/

Cloning into 'NetworkAnalysis_TwitterTrends'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 296 (delta 23), reused 33 (delta 16), pack-reused 247
Receiving objects: 100% (296/296), 32.74 MiB | 32.21 MiB/s, done.
Resolving deltas: 100% (110/110), done.


# Project


In [38]:
#@title Dependencias
import numpy as np
import networkx as nx

In [51]:
#@title ***[WARNING]***: Siempre se debe correr esta celda para tener los mismo resultados.  
import numpy as np
from burstkit.util import SEED
np.random.seed(SEED)

In [15]:
import burstkit.util as util
from burstkit.util import read_files
import burstkit.util.read_files as rf

type(rf.get_timeline_retweets())

dict